## Classification for xenophobia/hate/racism (on immigrants)
### Datasets
[sakren/twitter_racism_dataset](https://huggingface.co/datasets/sakren/twitter_racism_dataset) 14K rows (85% none, 15% racism)

We found but did not implement these datasets:


Hatespeech data catalogue https://hatespeechdata.com/#English-header


SemEval-2019 Task5  https://aclanthology.org/S19-2007/


https://github.com/aymeam/Datasets-for-Hate-Speech-Detection?tab=readme-ov-file


## Propossed approach
Fine-tuning a pretrained model like BERT often yields the best results. Depending on our requirements, we can choose between different versions of the model, such as BERT-large or BERT-base. Alternatively, we can train an SVM model on the same dataset, achieving relatively good results with a less complex model compared to BERT-based models.

In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

c:\Users\nilif\OneDrive\Desktop\Task_Seraj\seraj_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset from the HF (5-fold cross validation)
vals_ds = load_dataset('sakren/twitter_racism_dataset', split=[
    f'train[{k}%:{k+10}%]' for k in range(0, 100, 20)
])
trains_ds = load_dataset('sakren/twitter_racism_dataset', split=[
    f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 20)
])


In [3]:
model = TFAutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize(batch):
    return tokenizer(batch["Text"], padding='max_length', truncation=True, max_length=128)


c:\Users\nilif\OneDrive\Desktop\Task_Seraj\seraj_env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
train_encoded = [t.map(tokenize, batched=True, batch_size=None) for t in trains_ds]
val_encoded = [t.map(tokenize, batched=True, batch_size=None) for t in vals_ds]

Map: 100%|██████████| 1347/1347 [00:00<00:00, 18617.15 examples/s]


In [8]:
len(train_encoded[0][0]['attention_mask'])

128

In [9]:
# setting BATCH_SIZE to 64.
BATCH_SIZE = 64

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, inp['oh_label']

def convert_data(train_ds, val_ds):
    # setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
    # to the tensorflow format. Now if you access this dataset you will get these
    # columns in `tf.Tensor` format
    train_ds.set_format('tf', 
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'oh_label'])

    # converting train split of the dataset to tensorflow format
    train_dataset = tf.data.Dataset.from_tensor_slices(train_ds[:])
    # set batch_size and shuffle
    train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
    # map the `order` function
    train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

    # ... doing the same for test set ...
    val_ds.set_format('tf', 
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'oh_label'])
    test_dataset = tf.data.Dataset.from_tensor_slices(val_ds[:])
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

    return train_dataset, test_dataset

In [10]:
class BERTForClassification(tf.keras.Model):
    
    def __init__(self, bert_model, num_classes, dropout_rate=0.5, hidden_units=128):
        super().__init__()
        self.bert = bert_model
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.hidden_units = hidden_units
        
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.hidden_layer1 = tf.keras.layers.Dense(hidden_units, activation='relu')
        self.hidden_layer2 = tf.keras.layers.Dense(hidden_units, activation='relu')
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs, training=False):
        x = self.bert(inputs)[1]
        x = self.hidden_layer1(x)
        x = self.dropout(x, training=training)
        x = self.hidden_layer2(x)
        x = self.dropout(x, training=training)
        return self.fc(x)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'bert_model': self.bert.name,
            'num_classes': self.num_classes,
            'dropout_rate': self.dropout_rate,
            'hidden_units': self.hidden_units
        })
        return config

    @classmethod
    def from_config(cls, config):
        bert_model = TFAutoModel.from_pretrained(config.pop('bert_model'))
        return cls(bert_model=bert_model, **config)

In [27]:
# 5-fold cross validation loop
history_list = []
classifier = None
for k in range(5):
    classifier = BERTForClassification(model, num_classes=2)

    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    train_dataset , test_dataset = convert_data(train_encoded[k], val_encoded[k])
    
    history = classifier.fit(
        train_dataset,
        epochs=1
    )

    classifier.evaluate(test_dataset)

    history_list.append(history)

    


22/22 [==============================] - 38s 2s/step - loss: 0.1304 - accuracy: 0.9480


In [12]:
classifier.summary()

Model: "bert_for_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model (TFBertModel  multiple                  109482240 
 )                                                               
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 dense (Dense)               multiple                  98432     
                                                                 
 dense_1 (Dense)             multiple                  16512     
                                                                 
 dense_2 (Dense)             multiple                  258       
                                                                 
Total params: 109597442 (418.08 MB)
Trainable params: 109597442 (418.08 MB)
Non-trainable params: 0 (0.00 Byt

In [13]:
# Define the input signature
input_signature = {
    'input_ids': tf.TensorSpec(shape=(None, 128), dtype=tf.int32, name='input_ids'),
    'attention_mask': tf.TensorSpec(shape=(None, 128), dtype=tf.int32, name='attention_mask'),
    'token_type_ids': tf.TensorSpec(shape=(None, 128), dtype=tf.int32, name='token_type_ids')
}

# Save the model with the input signature
classifier._saved_model_inputs_spec = input_signature

# saving the model
classifier.save('trained_model/bert_base_uncased_finetuned')

INFO:tensorflow:Assets written to: trained_model/bert_base_uncased_finetuned\assets


INFO:tensorflow:Assets written to: trained_model/bert_base_uncased_finetuned\assets


In [14]:
# loading the model
loaded_model = tf.keras.models.load_model('trained_model/bert_base_uncased_finetuned', custom_objects={'BERTForClassification': BERTForClassification, 'TFBertModel': TFAutoModel.from_pretrained("bert-base-uncased")})

c:\Users\nilif\OneDrive\Desktop\Task_Seraj\seraj_env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly 

In [16]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Example input text
input_text = "True. Stoning and beheading were much more to Mohammed's taste."

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors='tf', padding='max_length', truncation=True, max_length=128)

# Cast the inputs to int32
inputs = {key: tf.cast(tensor, tf.int64) for key, tensor in inputs.items()}


# Ensure 'token_type_ids' is included in the inputs
if 'token_type_ids' not in inputs:
    inputs['token_type_ids'] = tf.zeros_like(inputs['input_ids'])



# Make predictions
predictions = loaded_model(inputs, training=False)

# Print the model output
print(predictions)

tf.Tensor([[0.9198044  0.08019559]], shape=(1, 2), dtype=float32)
